In [4]:
import pandas as pd
import datetime
import pytz
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import plotly.io as pio

In [14]:
display(df)

,Datetime_CST,Date,Time,Open,High,Low,Close,Volume,Datetime,Percent_Change,Release_Datetime,Time_to_Release_Minutes
0,2011-03-09 08:30:00-06:00,09/03/2011,08:30:00,105.56,105.56,105.44,105.46,514,2011-03-09 08:30:00,-0.094733,2011-03-09 09:30:00-06:00,-60.0
1,2011-03-09 08:31:00-06:00,09/03/2011,08:31:00,105.46,105.52,105.45,105.46,323,2011-03-09 08:31:00,0.000000,2011-03-09 09:30:00-06:00,-59.0
2,2011-03-09 08:32:00-06:00,09/03/2011,08:32:00,105.45,105.47,105.31,105.32,899,2011-03-09 08:32:00,-0.132752,2011-03-09 09:30:00-06:00,-58.0
3,2011-03-09 08:33:00-06:00,09/03/2011,08:33:00,105.31,105.37,105.25,105.33,716,2011-03-09 08:33:00,0.009495,2011-03-09 09:30:00-06:00,-57.0
4,2011-03-09 08:34:00-06:00,09/03/2011,08:34:00,105.34,105.45,105.31,105.40,660,2011-03-09 08:34:00,0.066458,2011-03-09 09:30:00-06:00,-56.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88339,2025-03-05 10:26:00-06:00,05/03/2025,10:26:00,65.42,65.46,65.37,65.38,713,2025-03-05 10:26:00,-0.061143,2025-03-05 09:30:00-06:00,56.0
88340,2025-03-05 10:27:00-06:00,05/03/2025,10:27:00,65.37,65.41,65.37,65.39,640,2025-03-05 10:27:00,0.015295,2025-03-05 09:30:00-06:00,57.0
88341,2025-03-05 10:28:00-06:00,05/03/2025,10:28:00,65.40,65.42,65.33,65.36,796,2025-03-05 10:28:00,-0.045879,2025-03-05 09:30:00-06:00,58.0
88342,2025-03-05 10:29:00-06:00,05/03/2025,10:29:00,65.35,65.40,65.35,65.37,658,2025-03-05 10:29:00,0.015300,2025-03-05 09:30:00-06:00,59.0


In [19]:

df=pd.read_csv("price_window.csv")
valid_releases = df.loc[df['Time_to_Release_Minutes'] == 2]
display(valid_releases['Date'])
display(df['Date'])
#Keep all rows from those groups
df = df[df['Date'].isin(valid_releases['Date'])]
df = df[df['Time_to_Release_Minutes'] <= 2]
display(df)
df.to_csv("price_window_valid.csv", index=False)

62       09/03/2011
183      16/03/2011
304      23/03/2011
425      30/03/2011
546      06/04/2011
            ...    
87801    05/02/2025
87922    12/02/2025
88043    20/02/2025
88164    26/02/2025
88285    05/03/2025
Name: Date, Length: 731, dtype: object

0        09/03/2011
1        09/03/2011
2        09/03/2011
3        09/03/2011
4        09/03/2011
            ...    
88339    05/03/2025
88340    05/03/2025
88341    05/03/2025
88342    05/03/2025
88343    05/03/2025
Name: Date, Length: 88344, dtype: object

,Datetime_CST,Date,Time,Open,High,Low,Close,Volume,Datetime,Percent_Change,Release_Datetime,Time_to_Release_Minutes
0,2011-03-09 08:30:00-06:00,09/03/2011,08:30:00,105.56,105.56,105.44,105.46,514,2011-03-09 08:30:00,-0.094733,2011-03-09 09:30:00-06:00,-60.0
1,2011-03-09 08:31:00-06:00,09/03/2011,08:31:00,105.46,105.52,105.45,105.46,323,2011-03-09 08:31:00,0.000000,2011-03-09 09:30:00-06:00,-59.0
2,2011-03-09 08:32:00-06:00,09/03/2011,08:32:00,105.45,105.47,105.31,105.32,899,2011-03-09 08:32:00,-0.132752,2011-03-09 09:30:00-06:00,-58.0
3,2011-03-09 08:33:00-06:00,09/03/2011,08:33:00,105.31,105.37,105.25,105.33,716,2011-03-09 08:33:00,0.009495,2011-03-09 09:30:00-06:00,-57.0
4,2011-03-09 08:34:00-06:00,09/03/2011,08:34:00,105.34,105.45,105.31,105.40,660,2011-03-09 08:34:00,0.066458,2011-03-09 09:30:00-06:00,-56.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88281,2025-03-05 09:28:00-06:00,05/03/2025,09:28:00,66.18,66.28,66.16,66.27,486,2025-03-05 09:28:00,0.135993,2025-03-05 09:30:00-06:00,-2.0
88282,2025-03-05 09:29:00-06:00,05/03/2025,09:29:00,66.27,66.31,66.19,66.22,587,2025-03-05 09:29:00,-0.075449,2025-03-05 09:30:00-06:00,-1.0
88283,2025-03-05 09:30:00-06:00,05/03/2025,09:30:00,66.21,66.32,66.10,66.22,940,2025-03-05 09:30:00,0.000000,2025-03-05 09:30:00-06:00,0.0
88284,2025-03-05 09:31:00-06:00,05/03/2025,09:31:00,66.22,66.32,66.22,66.25,644,2025-03-05 09:31:00,0.045304,2025-03-05 09:30:00-06:00,1.0


In [45]:
def pivot_market_data(df, x_minutes_before, cols_to_pivot=None):
    """
    pivot into wide formate where minutes to release from [-x, 2] are kept as columns for each feature.
    """
    if cols_to_pivot is None:
        cols_to_pivot = ['Open', 'High', 'Low', 'Close', 'Volume']

    df_filtered = df[(df['Time_to_Release_Minutes'] <= 2) & (df['Time_to_Release_Minutes'] >= -x_minutes_before)].copy()

    df_long = df_filtered.melt(
        id_vars=['Datetime', 'Time_to_Release_Minutes', 'Release_Datetime'],
        value_vars=cols_to_pivot,
        var_name='Feature',
        value_name='Value'
    )
    df_long['Feature_min'] = df_long['Feature'] + '_t' + df_long['Time_to_Release_Minutes'].astype(int).astype(str)

    df_wide = df_long.pivot_table(
        index='Release_Datetime',
        columns='Feature_min',
        values='Value'
    ).reset_index()
    cols_to_drop = [col for col in df_wide.columns if 't2' in col and col != 'Close_t2']
    df_wide.drop(columns=cols_to_drop, inplace=True)
    return df_wide

df_wide = pivot_market_data(df, x_minutes_before=60)

df_wide['Date'] = df_wide.apply(lambda x: x['Release_Datetime'].split()[0], axis=1)

display(df_wide.isna().sum().where(df_wide.isna().sum() > 0))

display(df_wide)

df_wide.to_csv("price_window_valid_wide.csv", index=False)



Feature_min
Release_Datetime   NaN
Close_t-1          NaN
Close_t-10         NaN
Close_t-11         NaN
Close_t-12         NaN
                    ..
Volume_t-8         NaN
Volume_t-9         NaN
Volume_t0          NaN
Volume_t1          NaN
Date               NaN
Length: 313, dtype: float64

Feature_min,Release_Datetime,Close_t-1,Close_t-10,Close_t-11,Close_t-12,Close_t-13,Close_t-14,Close_t-15,Close_t-16,Close_t-17,...,Volume_t-58,Volume_t-59,Volume_t-6,Volume_t-60,Volume_t-7,Volume_t-8,Volume_t-9,Volume_t0,Volume_t1,Date
0,2011-03-09 09:30:00-06:00,105.13,105.15,105.14,105.17,105.19,105.20,105.20,105.33,105.20,...,899.0,323.0,318.0,514.0,138.0,254.0,252.0,2458.0,2052.0,2011-03-09
1,2011-03-16 08:30:00-05:00,98.38,98.44,98.35,98.39,98.35,98.35,98.43,98.53,98.63,...,127.0,232.0,275.0,401.0,491.0,262.0,190.0,588.0,473.0,2011-03-16
2,2011-03-23 08:30:00-05:00,105.36,105.26,105.21,105.19,105.15,105.12,105.02,105.03,105.04,...,350.0,121.0,111.0,187.0,187.0,466.0,412.0,325.0,983.0,2011-03-23
3,2011-03-30 08:30:00-05:00,104.34,104.35,104.36,104.36,104.42,104.39,104.40,104.40,104.40,...,119.0,201.0,133.0,484.0,382.0,548.0,188.0,280.0,292.0,2011-03-30
4,2011-04-06 08:30:00-05:00,108.62,108.48,108.52,108.55,108.52,108.54,108.53,108.59,108.59,...,29.0,26.0,265.0,37.0,199.0,182.0,86.0,556.0,567.0,2011-04-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2025-02-05 09:30:00-06:00,71.66,71.61,71.60,71.54,71.53,71.57,71.55,71.61,71.63,...,313.0,319.0,246.0,547.0,206.0,379.0,513.0,2069.0,929.0,2025-02-05
727,2025-02-12 09:30:00-06:00,72.10,72.04,72.09,72.11,72.07,72.07,72.05,72.02,72.05,...,188.0,202.0,159.0,491.0,109.0,183.0,140.0,1269.0,515.0,2025-02-12
728,2025-02-20 11:00:00-06:00,72.71,72.75,72.72,72.71,72.70,72.72,72.75,72.77,72.77,...,166.0,140.0,100.0,158.0,109.0,76.0,94.0,932.0,325.0,2025-02-20
729,2025-02-26 09:00:00-06:00,68.93,68.94,68.89,68.86,68.88,68.94,68.93,68.83,68.82,...,640.0,1314.0,160.0,989.0,226.0,263.0,260.0,477.0,349.0,2025-02-26


Starting with xgboost on just the price data as a baseline

In [33]:
df_wide.drop(columns=['Datetime']).columns

Index(['Close_t-1', 'Close_t-10', 'Close_t-11', 'Close_t-12', 'Close_t-13',
       'Close_t-14', 'Close_t-15', 'Close_t-16', 'Close_t-17', 'Close_t-18',
       ...
       'Volume_t-58', 'Volume_t-59', 'Volume_t-6', 'Volume_t-60', 'Volume_t-7',
       'Volume_t-8', 'Volume_t-9', 'Volume_t0', 'Volume_t1', 'Date'],
      dtype='object', name='Feature_min', length=312)

In [ ]:
price_window_wide = pd.read_csv('price_window_valid_wide.csv')

display(price_window_wide)


,Datetime,Close_t-1,Close_t-10,Close_t-11,Close_t-12,Close_t-13,Close_t-14,Close_t-15,Close_t-16,Close_t-17,...,Volume_t-58,Volume_t-59,Volume_t-6,Volume_t-60,Volume_t-7,Volume_t-8,Volume_t-9,Volume_t0,Volume_t1,Volume_t2
0,2011-03-09,105.13,105.15,105.14,105.17,105.19,105.20,105.20,105.33,105.20,...,899.0,323.0,318.0,514.0,138.0,254.0,252.0,2458.0,2052.0,1250.0
1,2011-06-02,100.31,100.47,100.48,100.38,100.51,100.47,100.44,100.56,100.62,...,410.0,466.0,193.0,412.0,153.0,120.0,178.0,252.0,354.0,297.0
2,2011-07-07,98.48,98.39,98.38,98.41,98.50,98.48,98.59,98.59,98.60,...,330.0,484.0,315.0,466.0,817.0,177.0,224.0,376.0,712.0,232.0
3,2011-09-08,89.60,89.57,89.55,89.52,89.52,89.48,89.45,89.61,89.64,...,332.0,598.0,145.0,650.0,505.0,142.0,230.0,169.0,383.0,271.0
4,2011-10-13,83.68,83.59,83.45,83.58,83.50,83.63,83.66,83.61,83.58,...,868.0,3401.0,388.0,883.0,256.0,229.0,274.0,364.0,944.0,562.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2025-01-29,73.35,73.35,73.36,73.36,73.48,73.42,73.51,73.59,73.67,...,246.0,199.0,448.0,627.0,193.0,270.0,127.0,1286.0,663.0,393.0
245,2025-02-05,71.66,71.61,71.60,71.54,71.53,71.57,71.55,71.61,71.63,...,313.0,319.0,246.0,547.0,206.0,379.0,513.0,2069.0,929.0,673.0
246,2025-02-12,72.10,72.04,72.09,72.11,72.07,72.07,72.05,72.02,72.05,...,188.0,202.0,159.0,491.0,109.0,183.0,140.0,1269.0,515.0,624.0
247,2025-02-26,68.97,69.04,69.03,69.07,69.08,69.07,69.01,68.96,68.96,...,292.0,364.0,131.0,495.0,77.0,195.0,195.0,828.0,717.0,580.0


In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_val)
mean_squared_error(y_val, y_pred)